# Train and save text classification model

### Import modules

In [1]:
import src.data.make_dataset as make_dataset
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [6]:
train_data, valid_data, classes = make_dataset.load_processed("../data/processed/")

In [2]:
train_data, valid_data, classes = make_dataset.load_data("../data/raw/")

INFO:root:Number of training samples:    3543746
INFO:root:Number of test     samples:     151860
INFO:root:Completed data shuffling.
INFO:root:Completed categories list.


In [7]:
train_data[0][15431]

'Hypocalcimia after thydroidectomy - Want new management?'

In [9]:
import src.models.train_model as train_model


In [11]:
train_model.train(train_data, valid_data, classes)

Loading Yahoo! Q&A data...
Loaded model 'en_core_web_sm'
Using 2000 examples (2000 training, 5000 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
20.005	0.545	0.020	0.039
5.555	0.538	0.044	0.082
1.986	0.459	0.108	0.175
0.736	0.420	0.172	0.245
0.306	0.408	0.220	0.286
0.143	0.391	0.247	0.303
0.071	0.381	0.265	0.313
0.045	0.370	0.273	0.314
0.021	0.364	0.281	0.317
0.011	0.359	0.285	0.318
0.007	0.355	0.289	0.319
0.005	0.353	0.295	0.321
0.003	0.351	0.296	0.321
0.005	0.349	0.299	0.322
0.002	0.345	0.297	0.319
0.002	0.345	0.301	0.321
0.002	0.347	0.305	0.324
0.002	0.349	0.308	0.327
0.001	0.348	0.309	0.327
0.001	0.343	0.305	0.323
This movie sucked {'Society & Culture': 3.4277999105825074e-08, 'Science & Mathematics': 1.4359703476209823e-13, 'Health': 7.139125401955937e-11, 'Education & Reference': 4.144410979324716e-16, 'Computers & Internet': 1.8301327120440192e-10, 'Sports': 5.0861096584364773e-11, 'Business & Finance': 4.719486696558306e-06, 'Entertainment & Music': 0.999992966651916

In [12]:
import src.models.predict_model as predict_model
predict_model.predict("Where to go to school.",model_dir="../models/")

Loading from ../models/
Where to go to school. {'Society & Culture': 1.1396258742024656e-05, 'Science & Mathematics': 5.580629476753529e-06, 'Health': 1.683458140178118e-05, 'Education & Reference': 0.8305782675743103, 'Computers & Internet': 2.7173664420843124e-05, 'Sports': 0.16575634479522705, 'Business & Finance': 0.0029712594114243984, 'Entertainment & Music': 0.0006290159071795642, 'Family & Relationships': 1.3134732625985635e-06, 'Politics & Government': 2.9151601665944327e-06}
